In [5]:
import nltk
import random
import json

In [6]:
with open('BIG_BOT_CONFIG.json','r') as f:
    BOT_CONFIG = json.load(f)

In [7]:
def cleaner(text):
    cleaned_text = ''
    for ch in text.lower():
        if ch in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя ':
            cleaned_text = cleaned_text + ch
    return cleaned_text


def match(text, example):
    return nltk.edit_distance(text, example) / len(example) < 0.4


def get_intent(text):
    for intent in BOT_CONFIG['intents']:
        for example in BOT_CONFIG['intents'][intent]['examples']:
            if match(cleaner(text), cleaner(example)):
                return intent


def bot(text):
    intent = get_intent(text)

    if intent is not None:
        return random.choice(BOT_CONFIG['intents'][intent]['responses'])
    else:
        return random.choice(BOT_CONFIG['default_answers'])


In [ ]:
quest = ''
while quest not in ['выход', "выключайся"]:
    quest = input()
    answer = bot(quest)
    print(answer)
